# Cigarette consumption in the U.S. by State - IV and OLS examples

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.gmm import IV2SLS
from sklearn.linear_model import LogisticRegression

### Data set consists of annual data for the 48 continental U.S. states.
* Quantity consumed is measured by annual per capita cigarette sales in packs per fiscal year, as derived from state tax collection data.
* The price is the average retail cigarette price per pack during the fiscal year, including taxes.
* The cigarette-specific tax is the tax applied to cigarettes only. 
* All prices, income, and taxes are deflated by the Consumer Price Index and thus are in constant (real) dollars.  

In [2]:
df = pd.read_excel('Clase 3/cig.xlsx')

In [3]:
df

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs
0,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335
1,AR,1985,1.076,2327000,128.534592,26210736,37.000000,101.474998,37.000000
2,AZ,1985,1.076,3184000,104.522614,43956936,31.000000,108.578751,36.170418
3,CA,1985,1.076,26444000,100.363037,447102816,26.000000,107.837341,32.104000
4,CO,1985,1.076,3209000,112.963539,49466672,31.000000,94.266663,31.000000
...,...,...,...,...,...,...,...,...,...
91,VT,1995,1.524,582827,122.334755,12448607,44.000000,175.638748,52.363750
92,WA,1995,1.524,5431024,65.530922,129680832,80.500000,239.109344,96.142670
93,WI,1995,1.524,5137004,92.466347,115959680,62.000000,201.381256,71.589584
94,WV,1995,1.524,1820560,115.568832,32611268,41.000000,166.517181,50.425499


**Columns description:**
* State: 48 continental U.S. states 
* Year: 1985 and 1995
* CPI: Consumer Price Index (U.S.)   
* pop: State population
* packpc: Number of packs consumed per capita (it's importante to highlight that relative consumption to population size should be calculated)
* income: State personal income (nominal)
* tax: Average stat, federal and average local taxes for FY.
* avgprs: Average price during FY, including sales tax.
* taxs: Average taxes fro FY, including sales tax.

## How can we determine the optimal level of tax prices for cigarettes?

1. Re-express values for inflation indices

In [4]:
df['real_price'] = df['avgprs'] / df['cpi']     # ==> Relative price for cigarettes and consumer index products (real purchase power).
df['cigtax'] = df['tax'] / df['cpi']

2. Create sales tax and income per capita variables

In [5]:
df['sales_tax'] = (df['taxs'] - df['tax']) / df['cpi'] # ==> Discrimination of Sales Tax 
df['income_per_capita'] = df['income'] / df['pop'] * df['cpi']    # ==> Income per capita  

3. Compute correlations between real prices and cigarette and sales taxes. 

In [11]:
corr_tax = df['real_price'].corr(df['cigtax'])              # ==> 0.84 - HIGH correlation between price and taxes.
corr_sales_tax = df['real_price'].corr(df['sales_tax'])     # ==> 0.70 - Also HIGH correlation but with prices and sales tax.

In [13]:
corr_tax, corr_sales_tax

(0.8402331041290141, 0.7035011873032436)

The above calculation denotes a high correlation between prices and taxes, although lower correlation with sales tax.

In [7]:
df_1995 = df[df['year'] == 1995]

4. Calculate linear regression - **One stage Ordinary Leaste Squares (OLS)** - 
In this example, resulting coefficientes are interpreted as demand elasticity for cigarettes. 

In [14]:
lm = smf.ols('np.log(packpc) ~ np.log(real_price)',data=df_1995)    
                                                
fit_lm = lm.fit()
print(fit_lm.summary())

                            OLS Regression Results                            
Dep. Variable:         np.log(packpc)   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     31.41
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           1.13e-06
Time:                        00:59:23   Log-Likelihood:                 12.724
No. Observations:                  48   AIC:                            -21.45
Df Residuals:                      46   BIC:                            -17.71
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             10.3389      1

### *Partial conclusion => An increase in 1% of prices should cause a 1,21% decrease in demand of cigarettes.*

## In addition, we can calculate a linear regression model with instrumental variables as endogenous variables are involved - 2 Stage OLS

1. First Stage OLS:

In [15]:
lm2_1 = smf.ols('np.log(real_price) ~ sales_tax', data=df_1995)

fit_lm2_1 = lm2_1.fit()
print(fit_lm2_1.summary())   

                            OLS Regression Results                            
Dep. Variable:     np.log(real_price)   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     40.96
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           7.27e-08
Time:                        01:01:09   Log-Likelihood:                 46.435
No. Observations:                  48   AIC:                            -88.87
Df Residuals:                      46   BIC:                            -85.13
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6165      0.029    158.601      0.0

### *As expected by the results of calculated correlations, R2 is relatively high.  In univariate regression models, R2 tends to be similar with correlation.*

2. Second stage OLS

In [16]:
orig = np.log(df_1995['real_price']).values
fitted = fit_lm2_1.fittedvalues.values      
lm2_2 = smf.ols('np.log(packpc) ~ fitted', data=df_1995)
fit_lm2_2 = lm2_2.fit()
print(fit_lm2_2.summary())   

                            OLS Regression Results                            
Dep. Variable:         np.log(packpc)   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     8.277
Date:                Fri, 28 Aug 2020   Prob (F-statistic):            0.00607
Time:                        01:04:05   Log-Likelihood:                 4.2040
No. Observations:                  48   AIC:                            -4.408
Df Residuals:                      46   BIC:                           -0.6656
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.7199      1.801      5.396      0.0

### *Obtain corrected regression with revised elasticy for cigarettes. However, Standard Deviations are manually calculated.*

In [17]:
fit_lm2_2_iv = IV2SLS(np.log(df_1995['packpc']),fitted, instrument=df_1995['sales_tax']).fit()
print(fit_lm2_2_iv.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                 packpc   R-squared:                       0.996
Model:                         IV2SLS   Adj. R-squared:                  0.996
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Fri, 28 Aug 2020                                         
Time:                        01:04:24                                         
No. Observations:                  48                                         
Df Residuals:                      47                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9300      0.010     91.733      0.0

C:\Users\Julian Estandar\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
